In [39]:
import sqlite3

class DatabaseManager:
    def __init__(self, db_name="vault.db"):
            self.conn = sqlite3.connect(db_name)
            self.create_tables()

    def create_tables(self):
        cursor = self.conn.cursor()
        cursor.execute(''' CREATE TABLE IF NOT EXISTS users (
            user_id INTEGER PRIMARY KEY,
            name TEXT NOT NULL)''')

        cursor.execute('''CREATE TABLE IF NOT EXISTS items (
            item_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            origin_era TEXT NOT NULL,
            max_duration INTEGER NOT NULL,
            is_borrowed INTEGER DEFAULT 0)''')

        cursor.execute(''' CREATE TABLE IF NOT EXISTS borrowed_items (
            user_id INTEGER,
            item_id INTEGER,
            turns_borrowed INTEGER,
            PRIMARY KEY (user_id, item_id),
            FOREIGN KEY (user_id) REFERENCES users(user_id),
            FOREIGN KEY (item_id) REFERENCES items(item_id))''')

        self.conn.commit()


        # ADD USER

        def add_user(self, user_id: int, name: str):
            cursor = self.conn.cursor()
            cursor.execute("INSERT OR IGNORE INTO users (user_id, name) VALUES (?, ?)", (user_id, name))
            self.conn.commit()


        # ADD ITEM

        def add_item(self, name: str, origin_era: str, max_duration: int):
            cursor.execute("INSERT OT IGNORE INTO items (name, origin_era, max_duration) VALUES (?. ?, ?)",
                           (name, origin_era, max_duration))
            self.conn.commit()

        # BORROW ITEM

        def  borrow_item(self, user_id: int, item_name: str):
            cursor = self.conn.cursor()
            cursor.execute("SELECT item_id, is_borrowed FROM items WHERE name = ?", (item_name,))
            result = cursor.fetchone()
            if not result:
                print("Item not found.")
                return
            item_id, is_borrowed = result
            if is_borrowed:
                print("Item already borrowed.")
                return

            cursor.execute("UPDATE items SET is_borrowed = 1 WHERE item_id = ?", (item_id,))
            cursor.execute("INSERT INTO borrowed_items (user_id, item_id) VALUES (?, ?)", (user_id, item_id))
            self.conn.commit()
            print(f"Borrowed: {item_name}")


        # RETURN ITEM 

        def return_item(self, user_id: int, item_name: str):
            cursor = self.conn.cursor()
            cursor.execute("SELECT item_id FROM items WHERE name = ?", (item_name,))
            result = cursor.fetchone()
            if not result:
                print("Item not found.")
                return
            item_id = result[0]
            cursor.execute("DELETE FROM borrowed_items WHERE user_id = ? AND item_id = ?", (user_id, item_id))
            cursor.execute("UPDATE items SET is_borrowed = 0 WHERE item_id = ?", (item_id,))
            self.conn.commit()
            print(f"Returned: {item_name}")


        # VIEW ALL ITEMS

        def list_items(self):
            cursor = self.conn.cursor()
            cursor.execute("SELEDT name, origin_era, max_duration, is_borrowed FROM items")
            return cursor.fetchall()


        def advance_turn(self):
            cursor.execute("UPDATE borrowed_items SET turns_borrowed = turns_borrowed + 1")
            self.conn.commit()

            cursor.execute('''
                SELECT u.name, i.name, i.max_duration, b.turns_borrowed
                FROM borrowed_items b
                JOIN users u ON b.user_id = u.user_id
                JOIN items i ON b.item_id = i.item_id
            ''')
            results = cursor.fetchall()
            for user_name, item_name, max_dur, turns in results:
                if turns > max_dur:
                    print(f" TIME DISRUPTION; {user_name} held '{item_name}' too long!")

        def close(self):
            self.conn.close()


    class GameManager:
        def __init__(self, db: 'DatabaseManager'): 
            self.db = db
   
        def add_user(self, user_id: int, name: str):
            self.db.add_user(user_id, name)

        def add_item(self, name: str, era: str, max_duration: int):
            self.db.add_item(name, ear, max_duration)

        def borrow_item(self, user_if: int, item_name: str):
            self.db.borrow_item(user_id, item_name)

        def return_item(self, user_id: int, item_name: str):
            self.db.return_item(user_id, item_name)

        def next_turn(self):
            self.db.advance_turn()


    def main():
        db = DatabaseManager()
        gm = GameManager(db)
            
        db.add_user(1, "Anna the Time Traveler")
        db.add_item("Laser Sword", "Future", 3)
        db.add_item("Pharaoh's Amulet", "Ancient Egypt", 2)
        db.add_item("Steam Gun", "Industrial Era", 4)


        while True:
            print("\n --- Time Traveler's Vault ---")
            print("1. View Item")
            print("2. Borrow Item")
            print("3. Return Item")
            print("4. Advance Time")
            print("5. Exit")

            choice = input("Choose an option: ")

            if choice == "1":
                for name, era, dur, borrowed in db.list_items():
                    status = "Borrowed" if borrowed else "Available"
                    print(f"{nam} ({era}) - Duration: {dur} - {status}")

            elif choice == "2":
                item_name = input("Enter item name to borrow: ")
                gm.borrow_item(1, item_name)

            elif choice == "3":
                item_name = input("Enter item name to return: ")
                gm.return_item(1, item_name)

            elif choice == "4":
                gm.next_turn()

            elif choice =="5":
                db.close()
                print("Goodbye, Time Traveler!")
                break

            else:
                print("Invalid input.")


  if __name__ == "__main__":
         main()
        
    

IndentationError: unindent does not match any outer indentation level (<string>, line 175)

NameError: name 'main' is not defined